In [ ]:
!pip -q install nltk pandas

In [ ]:
import re
import unicodedata
import pandas as pd
from typing import List
from google.colab import drive
from copy import deepcopy


import nltk
from nltk.corpus import stopwords
from nltk.stem import RSLPStemmer

nltk.download("stopwords", quiet=True)
nltk.download("rslp", quiet=True)

drive.mount("/content/drive", force_remount=True)

In [ ]:
CSV_PATH = "/content/drive/MyDrive/M11/dados/all_msgs_whatsapp_instagram.csv"

df = pd.read_csv(CSV_PATH)
df = (
    df[["message"]]
    .dropna()
    .rename(columns={"message": "original"})
    .reset_index(drop=True)
)

In [ ]:
def to_lower(text: str) -> str:
    return text.lower()


def strip_accents(text: str) -> str:
    normalized = unicodedata.normalize("NFD", text)
    return "".join(ch for ch in normalized if unicodedata.category(ch) != "Mn")


PUNCT_PATTERN = re.compile(r"[\"'`.,;:!?()\[\]{}<>@#$%^&*_+=~/\\|-]")


def remove_punctuation(text: str) -> str:
    return PUNCT_PATTERN.sub(" ", text)


def simple_tokenize(text: str) -> List[str]:
    return [t for t in text.split() if t.strip()]


pt_stopwords = set(stopwords.words("portuguese"))


def remove_stopwords(tokens: List[str]) -> List[str]:
    return [t for t in tokens if t not in pt_stopwords]


stemmer = RSLPStemmer()


def stem_tokens(tokens: List[str]) -> List[str]:
    return [stemmer.stem(t) for t in tokens]

In [ ]:
df["lower"] = df["original"].apply(to_lower)
df["no_accents"] = df["lower"].apply(strip_accents)
df["no_punct"] = df["no_accents"].apply(remove_punctuation)
df["tokens"] = df["no_punct"].apply(simple_tokenize)
df["no_stopwords"] = df["tokens"].apply(remove_stopwords)
df["stems"] = df["no_stopwords"].apply(stem_tokens)
df.head(10)

In [ ]:
PIPELINE_STEPS = {
    "lower": True,
    "strip_accents": True,
    "remove_punctuation": True,
    "tokenize": True,
    "remove_stopwords": True,
    "stem": True,
}


def run_pipeline(text: str):
    x = text
    steps_result = {"original": text}
    if PIPELINE_STEPS["lower"]:
        x = to_lower(x)
        steps_result["lower"] = x
    if PIPELINE_STEPS["strip_accents"]:
        x = strip_accents(x)
        steps_result["no_accents"] = x
    if PIPELINE_STEPS["remove_punctuation"]:
        x = remove_punctuation(x)
        steps_result["no_punct"] = x
    if PIPELINE_STEPS["tokenize"]:
        x = simple_tokenize(x)
        steps_result["tokens"] = deepcopy(x)
    if PIPELINE_STEPS["remove_stopwords"]:
        x = remove_stopwords(x)
        steps_result["no_stopwords"] = deepcopy(x)
    if PIPELINE_STEPS["stem"]:
        x = stem_tokens(x)
        steps_result["stems"] = deepcopy(x)
    steps_result["final"] = x
    return steps_result


pipeline_df = pd.DataFrame([run_pipeline(s) for s in df["original"].tolist()])
pipeline_df.head(10)

In [ ]:
# Figura 1: diagrama do pipeline (gerado via networkx/matplotlib)
import matplotlib.pyplot as plt
import networkx as nx

stages = [
    "original",
    "lower",
    "no_accents",
    "no_punct",
    "tokens",
    "no_stopwords",
    "stems",
]
G = nx.DiGraph()
for i in range(len(stages) - 1):
    G.add_edge(stages[i], stages[i + 1])

pos = {
    "original": (0, 0),
    "lower": (1.5, 0),
    "no_accents": (3.0, 0),
    "no_punct": (4.5, 0),
    "tokens": (6.0, 0),
    "no_stopwords": (7.5, 0),
    "stems": (9.0, 0),
}

plt.figure(figsize=(12, 3.8))
nx.draw(
    G,
    pos,
    with_labels=True,
    node_color="#e6f2ff",
    node_size=8000,
    arrows=True,
    arrowstyle="->",
    arrowsize=15,
)
plt.title("Figura 1 - Pipeline de pré-processamento textual")
plt.axis("off")
plt.show()

print(
    "Legenda - Figura 1: Fluxo de estágios do pipeline do texto bruto (original) até a saída final (stems)."
)

In [ ]:
# Tabela 1: resumo de tamanho das mensagens (tokens)
import pandas as pd

lengths = pd.Series([len(t) for t in df["tokens"]])
lengths_summary = lengths.describe()
lengths_summary.to_frame(name="tokens_por_mensagem")

In [ ]:
# Gráfico 1: histograma de tamanho das mensagens (tokens)
import matplotlib.pyplot as plt

plt.figure(figsize=(8, 4))
plt.hist(lengths, bins=30, color="#4e79a7", edgecolor="white")
plt.title("Gráfico 1 - Distribuição do tamanho das mensagens (número de tokens)")
plt.xlabel("nº de tokens por mensagem")
plt.ylabel("frequência")
plt.show()
print(
    "Legenda - Gráfico 1: Histograma do número de tokens por mensagem após tokenização simples."
)

In [ ]:
# Tabela 2: palavras mais frequentes após stopwords + stem
from collections import Counter

stems_flat = [t for tokens in df["stems"] for t in tokens]
most_common = Counter(stems_flat).most_common(20)
mc_df = pd.DataFrame(most_common, columns=["termo", "frequencia"])
mc_df

In [ ]:
# Gráfico 2: top-20 termos após stopwords + stem
plt.figure(figsize=(10, 4))
plt.bar(mc_df["termo"], mc_df["frequencia"], color="#f28e2b")
plt.xticks(rotation=45, ha="right")
plt.title(
    "Gráfico 2 - Termos mais frequentes após remoção de stopwords e stemming (top-20)"
)
plt.xlabel("termo")
plt.ylabel("frequência")
plt.tight_layout()
plt.show()
print(
    "Legenda - Gráfico 2: Frequência dos 20 termos mais comuns após o pipeline final."
)

In [ ]:
# Tabela 3: estatísticas por etapa do pipeline (comprimento médio)
import numpy as np

stats = []
for col in ["original", "lower", "no_accents", "no_punct"]:
    stats.append(
        {"etapa": col, "comprimento_medio": df[col].astype(str).str.len().mean()}
    )
for col in ["tokens", "no_stopwords", "stems"]:
    stats.append({"etapa": col, "comprimento_medio": df[col].apply(len).mean()})
stats_df = pd.DataFrame(stats)
stats_df

In [ ]:
# Gráfico 3: comprimento médio por etapa do pipeline
plt.figure(figsize=(8, 4))
plt.bar(stats_df["etapa"], stats_df["comprimento_medio"], color="#59a14f")
plt.xticks(rotation=30, ha="right")
plt.title("Gráfico 3 - Comprimento médio por etapa do pipeline")
plt.xlabel("etapa")
plt.ylabel("comprimento médio")
plt.tight_layout()
plt.show()
print("Legenda - Gráfico 3: Comparação do comprimento médio entre etapas do pipeline.")